In [ ]:
# explore semantic search on telegram messages
# preprocess the data. e.g remove short texts
# try indexing the data with pinecone to check the latency (5000 messages?)

In [1]:
import os
import json
import sys
import pandas as pd
import numpy as np
import openai
import time
import re
import math
from telethon.sync import TelegramClient
from IPython.display import display
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import pinecone

from telethon.tl.types import InputMessagesFilterEmpty

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
pd.options.display.max_columns = None

# do not truncate column width in pandas
pd.options.display.max_colwidth = 200

/Users/camilo.ramirez/Library/Caches/pypoetry/virtualenvs/telegram-nlp-6m6KizWy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os

'/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp'

In [7]:
import sys
import os

In [3]:
sys.path

['/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp/notebooks',
 '/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp/notebooks',
 '//modules',
 '/Users/camilo.ramirez/.pyenv/versions/3.10.11/lib/python310.zip',
 '/Users/camilo.ramirez/.pyenv/versions/3.10.11/lib/python3.10',
 '/Users/camilo.ramirez/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload',
 '',
 '/Users/camilo.ramirez/Library/Caches/pypoetry/virtualenvs/telegram-nlp-6m6KizWy-py3.10/lib/python3.10/site-packages']

In [4]:
foo = "/Users/camilo.ramirez/Documents/mai-repos/telegram-nl/modules"
sys.path.insert(0, foo)

In [8]:
sys.path

['/Users/camilo.ramirez/Documents/mai-repos/telegram-nl/modules',
 '/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp/notebooks',
 '/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp/notebooks',
 '//modules',
 '/Users/camilo.ramirez/.pyenv/versions/3.10.11/lib/python310.zip',
 '/Users/camilo.ramirez/.pyenv/versions/3.10.11/lib/python3.10',
 '/Users/camilo.ramirez/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload',
 '',
 '/Users/camilo.ramirez/Library/Caches/pypoetry/virtualenvs/telegram-nlp-6m6KizWy-py3.10/lib/python3.10/site-packages']

In [9]:
os.environ["PYTHONPATH"]

'://modules/'

In [6]:
nb_dir = os.path.split(os.getcwd())[0]

In [9]:
os.path.split(os.getcwd())

('/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp', 'notebooks')

In [8]:
os.getcwd()

'/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp/notebooks'

In [7]:
nb_dir

'/Users/camilo.ramirez/Documents/mai-repos/telegram-nlp'

In [12]:
import modules.sentiment_predictor

ModuleNotFoundError: No module named 'modules'

In [2]:
api_id = os.environ["TELEGRAM_API_ID"]
api_hash = os.environ["TELEGRAM_API_HASH"]
pinecone_key = os.environ["PINECONE_APIKEY"]
phone = "+34634454832"
username = "@elvesipeto"
messages = []

In [19]:
pd_data = []

columns = ["channel_name", "id", "peer_id", "date", "message", "out", "mentioned",
        "media_unread", "silent", "post", "from_scheduled", "legacy", 
        "edit_hide", "pinned","noforwards", "from_id", "fwd_from", "via_bot_id",
        "reply_to", "media", "reply_markup", "entities", "views",
        "forwards", "replies", "edit_date", "post_author", "grouped_id",
        "reactions", "restriction_reason", "ttl_period"]

client = TelegramClient("session_temp", api_id, api_hash)
channel_id = "@runonflux"
n = 5000

async with client:        
    async for msg in client.iter_messages(channel_id, filter=InputMessagesFilterEmpty, limit=n):
        pd_data.append((channel_id, msg.id,msg.peer_id, msg.date, msg.message,
                msg.out, msg.mentioned, msg.media_unread, msg.silent,msg.post,
                msg.from_scheduled, msg.legacy, msg.edit_hide, msg.pinned, msg.noforwards,
                msg.from_id, msg.fwd_from, msg.via_bot_id, msg.reply_to, msg.media, msg.reply_markup,
                msg.entities, msg.views, msg.forwards, msg.replies, msg.edit_date, msg.post_author,
                msg.grouped_id, msg.reactions, msg.restriction_reason, msg.ttl_period
        ))

df = pd.DataFrame(pd_data, columns=columns)

In [29]:
# Data cleaning and filtering 

df["token_count"] = df["message"].apply(lambda x: len(x.split(" "))  if type(x) == str else 0)
print("Total message count", len(df))

filtered_df = df[~df["message"].isna()]
print("Total messages after excluding empty", len(filtered_df))

# What is the distribution of token counts?
display(filtered_df["token_count"].quantile([.1, .25, .5, .75, .95, 0.98]))


lower_limit = 4

# 250 was the max input lenght of the training data for multi-qa-MiniLM-L6-cos-v1
upper_limit = 250 

filtered_df = filtered_df[(filtered_df["token_count"] > lower_limit) & (filtered_df["token_count"] < upper_limit)]
print("Total messages after excluding long and short messages", len(filtered_df))

# clean messages
filtered_df.loc[:, "clean_message"] = filtered_df["message"].apply(lambda x: re.sub('[^A-Za-z0-9 .,?$%\'"]+', '', x))

Total message count 5000
Total messages after excluding empty 4849


0.10      1.0
0.25      4.0
0.50      9.0
0.75     19.0
0.95     57.6
0.98    135.0
Name: token_count, dtype: float64

Total messages after excluding long and short messages 3462


In [30]:
filtered_df.to_csv(f"data/{channel_id}.csv", index=False)

In [22]:
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key=os.environ["PINECONE_APIKEY"],
    environment="us-west1-gcp"
)

In [23]:
# embeddings model
multi_qa_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query = "This coin will moon soon"
vec = multi_qa_encoder.encode(query, convert_to_tensor=False)

In [33]:
FORCE_DELETE_INDEX = True
INDEX_NAME = "telegram-embeddings"

if FORCE_DELETE_INDEX:
    pinecone.delete_index(INDEX_NAME)

if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(INDEX_NAME, dimension=len(vec))
    
# connect to index
index = pinecone.Index(INDEX_NAME)

In [46]:
import torch

def get_device():
    has_gpu = torch.cuda.is_available()
    has_mps = torch.backends.mps.is_built()
    device = "mps" if has_mps else "gpu" if has_gpu else "cpu"
    return device

get_device()

'mps'

In [48]:
# Upload data to index

COMPUTE_EMBEDDINGS = True
if COMPUTE_EMBEDDINGS:
    # preprocess data
    df = pd.read_csv(f"data/{channel_id}.csv")

    # create embeddings
    df["embeddings"] = df["clean_message"].apply(lambda x: multi_qa_encoder.encode(x, device="mps", convert_to_tensor=True, show_progress_bar=False))
    
    index_upsert = []

df.head(1)

,channel_name,id,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,from_id,fwd_from,via_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period,token_count,clean_message,embeddings
0,@runonflux,492112,PeerChannel(channel_id=1202951585),2023-10-09 15:03:45+00:00,I agree. A level around $0.45/$0.50 would be a more bullish sign. Considering that at the beginning of the year this was also more or less the price,False,False,False,False,False,False,False,False,False,False,PeerUser(user_id=1879556613),NaN,NaN,"MessageReplyHeader(reply_to_msg_id=492105, reply_to_scheduled=False, forum_topic=True, reply_to_peer_id=None, reply_to_top_id=476033)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,I agree. A level around $0.45$0.50 would be a more bullish sign. Considering that at the beginning of the year this was also more or less the price,"[tensor(-0.0158, device='mps:0'), tensor(0.0359, device='mps:0'), tensor(-0.0107, device='mps:0'), tensor(0.0137, device='mps:0'), tensor(-0.0404, device='mps:0'), tensor(-0.0157, device='mps:0'),..."


#### Try encoding with Pytorch

In [58]:
telegram_messages = df["clean_message"].to_list()

In [62]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#Encode text
def encode(texts, device="cpu"):
    input_ids = []
    attention_masks = []
    
    # Tokenize sentences
    encoded_input = tokenizer(
        texts,
        max_length=64,
        padding='max_length', 
        truncation=True, 
        return_tensors='pt')
    
    

    input_ids = encoded_input["input_ids"].to(device)
    attention_masks = encoded_input["attention_mask"].to(device)

    model.to(device)

    # Compute token embeddings
    with torch.no_grad():        
        outputs = model(input_ids, attention_mask=attention_masks)

    # Perform pooling
    embeddings = mean_pooling(outputs, attention_mask=attention_masks)

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Sentences we want sentence embeddings for
query = "What is the most profitable node?"
docs = telegram_messages

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

#Encode query and docs
query_emb = encode(query, device="cpu")
doc_emb = encode(docs, device="cpu")


#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs[:10]:
    print(score, doc)


0.8341584205627441 A node is more profitable i guess
0.8178715705871582 How much are nodes still profitable?
0.779725193977356 oh yeah sorry I didn't mean nodes are not any more profitable
0.7323274612426758 Still profitable. 70$ cost, 140$ rewards.Nobody is saying that it wasn't more at some point, it was also less at some point.But nodes are profitable
0.6950095891952515 If some nodes close down, the rest becomes more profitable. It equals itself out.
0.6212084889411926 That was an answer to a different question.Also nodes are still profitable. My stratus costs around 70$ to run and gets about 130$ in rewards. Without future pa
0.5766648054122925 its strict benchmark, not all average person can run a node.
0.5579705238342285 Margins are tight but still profitable if you run it from home and already pay for internet regardless of nodes
0.5410272479057312 your setup much be inefficient  my nodes are profitable
0.5327602028846741 But I see I should make research what is most profitable


In [52]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Sentences we want sentence embeddings for
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

#Encode query and docs
query_emb = encode(query)
doc_emb = encode(docs)

#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)


0.9156372547149658 Around 9 Million people live in London
0.4947578012943268 London is known for its financial district


#### Upload eembeddings

In [44]:
import math
UPLOAD_VECTORS = True
if UPLOAD_VECTORS:    
    batch_size = 1000
    total_batches = math.ceil(len(df) / batch_size)
    start = 0

    for i in range(total_batches):
        index_upsert = [] # always initialize for each batch        
        end = start + batch_size

        print(f"iterating messages {start}-{end}")
        for i, item in df[start:end].iterrows():
            index_upsert += [
                    (str(i), 
                    item["embeddings"].tolist(),
                    {
                        "clean_message": item["clean_message"],
                        "channel_name": item["channel_name"],
                        "messagee_id": item["id"]
                    })
            ]
        start = end
        print(f"inserting batch {i}")
        index.upsert(vectors=index_upsert) # can contain maximum 1000 items        

iterating messages 0-1000
inserting batch 999
iterating messages 1000-2000
inserting batch 1999
iterating messages 2000-3000
inserting batch 2999
iterating messages 3000-4000
inserting batch 3461
